In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [ ]:
%matplotlib inline
%config InlienBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

In [ ]:
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("Monthly Sales Data.csv")

In [ ]:
keep_columns=['RSXFSN']
df=df[keep_columns]
#rescaling the values
df=df/10000

In [ ]:
#2. Set Training & Test Data (80:20)
n_train = int(len(df) * 0.8)
n_test = len(df) - n_train
train, test = df.iloc[0:n_train], df.iloc[n_train:len(df)]
print(len(train),len(test))



In [ ]:
#3. Convert to Time Series Data
def get_timeseries(x, n_steps=1):
    x_ts, y_ts = [], []
    for ii in range(len(x) - n_steps):
        v = x.iloc[ii:(ii + n_steps)].values
        x_ts.append(v)
        w = x.iloc[ii + n_steps].values
        y_ts.append(w)
    return np.array(x_ts), np.array(y_ts)


In [ ]:
n_steps = 12
n_features=1
x_train, y_train = get_timeseries(train, n_steps)
x_test, y_test = get_timeseries(test,n_steps)

print(x_train.shape,y_train.shape)

In [ ]:
#4. Setup Neural Network
from keras.optimizers import Adam

model = Sequential()
model.add(LSTM(units=16, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(1))

print(model.summary())

#Set the training parameters: loss and the optimizer
model.compile(loss='mean_squared_error',optimizer=Adam(0.001))

In [ ]:
#5. Fit Model
history = model.fit(x_train, y_train,epochs=500,batch_size=32,validation_split=0.1)

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss over Epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
#6. Evaluation (Training Data)
#plt.plot(np.arange(0,len(y_train)),'g',label='History')
plt.plot(np.arange(len(y_train),len(y_train)+len(y_test)),y_test,marker='.',label='Actual Data')
plt.plot(np.arange(len(y_train),len(y_train)+len(y_test)),y_pred,'r',label='Prediction')
plt.title('Training: Actual vs Predicted')
plt.ylabel('Retail Sales')
plt.xlabel('Period')
plt.legend()
plt.show()

In [ ]:
#7. Evaluation (Test Data)

plt.plot(y_test, marker='.', label="Actual Data")
plt.plot(y_pred, 'r', label="Prediction")
plt.title('Testing: Actual vs Predicted')
plt.ylabel('Retail Sales')
plt.xlabel('Period')
plt.legend()
plt.show();